In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split


In [3]:
df = pd.read_csv("features.csv", sep=";")
df = df.drop('semanticobjscore', axis=1) 
df = df.drop('semanticsubjscore', axis=1) 
df = df.drop('URL', axis=1) 

In [ ]:
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split


X = df.drop(['Label', 'TextID'], axis=1)  
y = df['Label']

# Convertir les labels en nombres
le = LabelEncoder()
y_encoded = le.fit_transform(y)

print("Classes:", le.classes_)  # Pour voir la correspondance

# Refaire la séparation train/test/val avec les labels encodés
X_train, X_test, y_train, y_test = train_test_split(
    X, y_encoded,
    test_size=0.2,          
    random_state=3,         
    stratify=y_encoded      
)

Classes: ['objective' 'subjective']


La fonction MLPClassifier prend par défaut ReLu comme fonction d'activation et la fonction sigmoïde pour la couche de sortie (car on est dans un cas binaire). L'algorithme par défaut est l'algorithme Adam (avec taux d'apprentissage 0,001 et taux de régularisation 0,0001)

LÀ ON VA UTILISER RANDOMSEARCH POUR CHOISIR LES HYPERPARAMÈTRES AU MIEUX 

In [ ]:
from sklearn.model_selection import RandomizedSearchCV
from sklearn.neural_network import MLPClassifier
from scipy.stats import loguniform, uniform, randint # Pour définir des distributions
from sklearn.metrics import precision_score 
from sklearn.preprocessing import StandardScaler



scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)


model_base = MLPClassifier(max_iter=2000, random_state=3)



# Nous définissons l'espace de recherche (le "param_dist")
param_dist = {
    # 1. Architecture (hidden_layer_sizes) : C'est une liste de tuples à choisir
    'hidden_layer_sizes': [(50,), (100,), (50, 25), (100, 50), (128, 64)], 
    
    # 2. Régularisation (alpha) : Loguniform est idéal pour les paramètres de régularisation
    'alpha': loguniform(1e-5, 1e-2), # Exemple: choisir une valeur entre 0.00001 et 0.01
    
    # 3. Optimisation (learning_rate_init) : Loguniform est aussi bon pour le taux d'apprentissage
    'learning_rate_init': loguniform(1e-4, 1e-2), # Exemple: choisir une valeur entre 0.0001 et 0.01
    
    # 4. Fonction d'activation : Une liste de choix discrets
    'activation': ['tanh', 'relu'],
    
    # 5. Type de solveur : Une liste de choix discrets
    'solver': ['adam', 'sgd'] 
}


# Instanciation de RandomizedSearchCV
# n_iter=50 : Le nombre total d'itérations aléatoires que nous voulons tester (compromis temps/précision)
# scoring='f1' : La métrique sur laquelle nous arbitrons (maximiser le F1-score)
# cv=5 : Utilise la validation croisée K-Fold avec 5 plis (plus fiable qu'un simple split Train/Val)
# verbose=2 : Affiche le progrès détaillé
# n_jobs=-1 : Utilise tous les cœurs du processeur pour accélérer le calcul
random_search = RandomizedSearchCV(
    estimator=model_base,
    param_distributions=param_dist,
    n_iter=50, 
    scoring='precision', 
    cv=5, 
    random_state=3, 
    verbose=2,
    n_jobs=-1 
)

# X_train et y_train sont les données utilisées pour la recherche d'hyperparamètres.
# L'objet random_search effectue l'entraînement et la validation croisée en interne.
print("Début de la recherche aléatoire...")
random_search.fit(X_train_scaled, y_train) 
print("Recherche aléatoire terminée.")


# Récupération du meilleur jeu d'hyperparamètres (celui qui a maximisé le Score de précision CV)
best_params = random_search.best_params_

# Récupération du modèle qui a obtenu le meilleur score de validation croisée
best_model = random_search.best_estimator_

print("\n---------------------------------------------")
print("MEILLEURS HYPERPARAMÈTRES TROUVÉS (JUSTIFICATION) :")
print(best_params)
print(f"Meilleur score de validation croisée (Score de précision) : {random_search.best_score_:.4f}")
print("---------------------------------------------")

# Évaluation finale sur l'ensemble de TEST
y_pred_test = best_model.predict(X_test_scaled)
test_precision = precision_score(y_test, y_pred_test)
print(f"Performance finale sur la base de TEST : Score de précision = {test_precision:.4f}")